# Selector Testing Notebook

This notebook is for experimenting with XPath/CSS selectors for ChatGPT, LinkedIn, and Twitter/X.
It launches a Chrome instance with a saved user profile and tests actions like typing, copying, and clicking.
All interactions are logged to verify selector accuracy.

In [ ]:
import undetected_chromedriver as uc
import os
import stat
import tempfile
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
try:
    import pyperclip
except ImportError:
    print("pyperclip not installed. Clipboard copying will be skipped.")
    pyperclip = None

# Chrome paths
CHROME_BINARY = r"C:\Users\abhay\Downloads\chrome-win64\chrome-win64\chrome.exe"
CHROMEDRIVER_SRC = r"C:\Users\abhay\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"

# User profile directory
USER_PROFILE_DIR = r"C:\Users\abhay\Desktop\bots\profiles\ubhay"

# Selector definitions (XPath or CSS, add more as needed)
SELECTORS = {
    'chatgpt': {
        'input_field': {'xpath': "/html/body/div[1]/div/div[1]/div[2]/main/div/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/div[1]/div[1]/div/div/div", 'css': ''},
        'output_div': {'xpath': "//div[@data-testid='conversation-turn-1']/div/div/div/div/div[1]/div", 'css': ''},
        'copy_button': {'xpath': "//div[@data-testid='conversation-turn-1']/div/div/div/div/div[2]/div/div/button[1]", 'css': ''}
    },
    'linkedin': {
        'post_input': {'xpath': "//div[@role='textbox']", 'css': ''},
        'post_button': {'xpath': "//button[contains(text(), 'Post')]", 'css': ''}
    },
    'twitter': {
        'tweet_input': {'xpath': "", 'css': ''},
        'tweet_button': {'xpath': "", 'css': ''}
    }
}

def patch_and_get_driver_path():
    """Copy chromedriver to a temp location and make it executable."""
    dst_fd, dst_path = tempfile.mkstemp(suffix=".exe")
    with open(CHROMEDRIVER_SRC, "rb") as src_f, open(dst_fd, "wb") as dst_f:
        dst_f.write(src_f.read())
    os.chmod(dst_path, stat.S_IWRITE | stat.S_IREAD | stat.S_IEXEC)
    return dst_path

def launch_browser():
    """Launch a Chrome instance with the specified user profile."""
    os.makedirs(USER_PROFILE_DIR, exist_ok=True)
    options = uc.ChromeOptions()
    options.binary_location = CHROME_BINARY
    options.add_argument(f"--user-data-dir={USER_PROFILE_DIR}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    
    driver_path = patch_and_get_driver_path()
    driver = uc.Chrome(driver_executable_path=driver_path, options=options)
    return driver

def get_element(driver, selector_type, selector_dict, timeout=10):
    """Flexible element finder for XPath or CSS selectors."""
    by_type = By.XPATH if selector_type == 'xpath' else By.CSS_SELECTOR
    selector = selector_dict[selector_type]
    if not selector:
        print(f"No {selector_type} selector defined.")
        return None
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable((by_type, selector))
        )
        print(f"Found element with {selector_type}: {selector}")
        return element
    except TimeoutException:
        print(f"Failed to find element with {selector_type}: {selector}")
        return None

def open_tabs(driver):
    """Open ChatGPT, LinkedIn, and Twitter in separate tabs."""
    # Tab 0: ChatGPT
    driver.get("https://chatgpt.com/")
    print(f"Tab 0 opened: ChatGPT (Handle: {driver.current_window_handle})")
    time.sleep(2)
    
    # Tab 1: LinkedIn
    driver.switch_to.new_window('tab')
    driver.get("https://www.linkedin.com/feed/")
    print(f"Tab 1 opened: LinkedIn (Handle: {driver.current_window_handle})")
    time.sleep(2)
    
    # Tab 2: Twitter/X
    driver.switch_to.new_window('tab')
    driver.get("https://twitter.com/home")
    print(f"Tab 2 opened: Twitter/X (Handle: {driver.current_window_handle})")
    time.sleep(2)
    
    # Switch back to ChatGPT tab
    driver.switch_to.window(driver.window_handles[0])
    print(f"Switched back to Tab 0: ChatGPT (Handle: {driver.current_window_handle})")

def test_chatgpt_interaction(driver, prompt="Test prompt: Write a short sentence."):
    """Test ChatGPT input, output, and copying."""
    driver.switch_to.window(driver.window_handles[0])
    driver.get("https://chatgpt.com/")
    
    # Test input field
    input_field = get_element(driver, 'xpath', SELECTORS['chatgpt']['input_field'])
    if input_field:
        input_field.clear()
        input_field.send_keys(prompt)
        input_field.send_keys(Keys.RETURN)
        print(f"Sent prompt: {prompt}")
    else:
        print("Input field test failed.")
        return
    
    # Wait for response
    output_div = get_element(driver, 'xpath', SELECTORS['chatgpt']['output_div'], timeout=30)
    if output_div:
        direct_output = output_div.text
        print("=== Direct Output (from div) ===")
        print(direct_output)
        print("================================")
    else:
        print("Output div test failed.")
        return
    
    # Test copy button
    copy_button = get_element(driver, 'xpath', SELECTORS['chatgpt']['copy_button'])
    if copy_button and pyperclip:
        copy_button.click()
        clipboard_output = pyperclip.paste()
        print("=== Clipboard Output (from copy button) ===")
        print(clipboard_output)
        print("================================")
    else:
        print("Copy button test failed or pyperclip not available.")

# Launch Chrome and open tabs
driver = launch_browser()
open_tabs(driver)

# Test ChatGPT interaction
test_chatgpt_interaction(driver)

# Keep browser open for manual inspection (comment out to auto-close)
# driver.quit()